<a href="https://colab.research.google.com/github/inesgoddi/textLevelClassifier/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import torch
import numpy as np
from transformers import BertTokenizer, BertModel
from torch import nn
from torch.optim import Adam
from tqdm import tqdm
from transformers import BertForSequenceClassification


In [ ]:
# Install CUDA Toolkit
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-11-0_11.0.3-1_amd64.deb
!dpkg -i cuda-11-0_11.0.3-1_amd64.deb
!apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda

# Install cuDNN (adjust version accordingly)
!wget https://developer.download.nvidia.com/compute/redist/cudnn/v8.0.5/cudnn-11.0-linux-x64-v8.0.5.39.tgz
!tar -xzvf cudnn-11.0-linux-x64-v8.0.5.39.tgz
!cp cuda/include/cudnn*.h /usr/local/cuda/include
!cp cuda/lib64/libcudnn* /usr/local/cuda/lib64
!chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*

--2024-07-08 00:18:01--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-11-0_11.0.3-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2446 (2.4K) [application/x-deb]
Saving to: ‘cuda-11-0_11.0.3-1_amd64.deb’

cuda-11-0_11.0.3-1_ 100%[===================>]   2.39K  --.-KB/s    in 0s      

2024-07-08 00:18:01 (208 MB/s) - ‘cuda-11-0_11.0.3-1_amd64.deb’ saved [2446/2446]

Selecting previously unselected package cuda-11-0.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack cuda-11-0_11.0.3-1_amd64.deb ...
Unpacking cuda-11-0 (11.0.3-1) ...
dpkg: dependency problems prevent configuration of cuda-11-0:
 cuda-11-0 depends on cuda-runtime-11-0 (>= 11.0.3); however:
  Package cuda-runtime-11-0 is not instal

In [ ]:
!nvidia-smi

Sun Jul  7 18:11:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
!pip install transformers --upgrade




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
ERROR: Operation cancelled by user


In [ ]:
import chardet

In [ ]:
from sklearn.model_selection import train_test_split

import pandas as pd

# Read the first Excel file into a pandas DataFrame
file1 = "debutants.xlsx"
df1 = pd.read_excel(file1)

# Read the second Excel file into a pandas DataFrame
file2 = "expert.xlsx"
df2 = pd.read_excel(file2)

# Merge or concatenate the DataFrames
# For example, if you want to concatenate them row-wise:
merged_df = pd.concat([df1, df2], ignore_index=True)
# Assuming your data is stored in X (features) and y (labels) lists or arrays
X_train, X_val, y_train, y_val = train_test_split(merged_df.iloc[:, 0], merged_df.iloc[:, 1], test_size=0.2, random_state=42)


In [ ]:
# Define column names
column_names = ['text', 'label']  # Add column names as needed

# Assign column names directly to the merged DataFrame
merged_df.columns = column_names

# Print the merged DataFrame to verify the header
print(len(merged_df.columns))

2


In [ ]:
test_datapath = f'test.csv'
#df = pd.read_csv(datapath,index_col=False)
#df.reset_index(drop=True, inplace=True)
with open('test.csv', 'rb') as f:
    encoding_test = chardet.detect(f.read())['encoding']
test_df = pd.read_csv(test_datapath, encoding=encoding_test)


In [ ]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', output_attentions=True, attn_implementation="eager")

labels = {
    '0': 0,
    '1': 1
}

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df, max_length=512, chunking_strategy="longest_first"):
        self.max_length = max_length
        self.chunking_strategy = chunking_strategy

        self.labels = [labels[label] for label in df['label'].astype(str)]
        self.texts = []

        for text in df['text']:
            # Tokenize and chunk the text into manageable parts
            tokenized_text = tokenizer.encode_plus(
                text,
                padding='max_length',
                max_length=self.max_length,
                truncation=True,
                return_tensors="pt",
                return_overflowing_tokens=True,
                stride=self.max_length - 128  # Adjust the stride as needed
            )

            # Extract individual chunks from tokenized_text
            for i in range(len(tokenized_text['input_ids'])):
                chunk = {}
                for key in tokenized_text.keys():
                    chunk[key] = tokenized_text[key][i]
                self.texts.append(chunk)

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        return self.texts[idx]

    def __getitem__(self, idx):
        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased', output_attentions=True, attn_implementation="eager")
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)
        self.cls_embedding = None
        #self.attentions = None


    def forward(self, input_ids, attention_mask):

        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        self.cls_embedding = outputs.last_hidden_state[:, 0, :]  # Extract CLS token embedding (first token)
        #self.attentions = outputs.attentions  # Store attentions for visualization

        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [ ]:
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)


    train_dataloader = torch.utils.data.DataLoader(train, batch_size=1, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()
    cls_embeddings_with_text = []
    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0
            num_batches = len(train_dataloader)


            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].squeeze(1).to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                cls_embedding = model.cls_embedding
                #attentions = model.attentions

                labels = [label for label in train_label]
                decoded_texts = [tokenizer.decode(ids,skip_special_tokens=True) for ids in input_id]

                for text,embedding,label in zip(decoded_texts,cls_embedding,labels):
                  cls_embeddings_with_text.append((text,embedding,label))

                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()

                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()

            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].squeeze(1).to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()

                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc

            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} | Train Accuracy: {total_acc_train / len(train_data): .3f} | Val Loss: {total_loss_val / len(val_data): .3f} | Val Accuracy: {total_acc_val / len(val_data): .3f}')
    return cls_embeddings_with_text

In [ ]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    cls_embeddings = []

    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].squeeze(1).to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              cls_embedding = model.cls_embedding
              #attentions = model.attentions


              labels = [label for label in test_label]
              decoded_texts = [tokenizer.decode(ids,skip_special_tokens=True) for ids in input_id]

              for text,embedding,label in zip(decoded_texts,output,labels):
                cls_embeddings.append((text,cls_embedding,label))


              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc


    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    return cls_embeddings

In [ ]:
np.random.seed(112)
train_df, val_df, test_df_2 = np.split(merged_df.sample(frac=1, random_state=42),
                                     [int(.8*len(merged_df)), int(.9*len(merged_df))])

print(len(train_df),len(val_df), len(test_df))


95 12 18


In [ ]:
EPOCHS = 10
num_classes = 2
model = BertClassifier()
LR = 2e-5

cls_embeddings_with_text = train(model, train_df, val_df, LR, EPOCHS)


100%|██████████| 95/95 [00:12<00:00,  7.82it/s]


Epochs: 1 | Train Loss:  0.162 | Train Accuracy:  0.947 | Val Loss:  0.024 | Val Accuracy:  1.000


100%|██████████| 95/95 [00:12<00:00,  7.73it/s]


Epochs: 2 | Train Loss:  0.015 | Train Accuracy:  1.000 | Val Loss:  0.008 | Val Accuracy:  1.000


100%|██████████| 95/95 [00:12<00:00,  7.76it/s]


...

Epochs: 8 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.001 | Val Accuracy:  1.000


100%|██████████| 95/95 [00:12<00:00,  7.83it/s]


Epochs: 9 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.001 | Val Accuracy:  1.000


100%|██████████| 95/95 [00:12<00:00,  7.83it/s]


Epochs: 10 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.001 | Val Accuracy:  1.000


In [ ]:
cls_embeddings = evaluate(model, test_df)

Test Accuracy:  0.889


In [ ]:

np.random.seed(112)
train_df, val_df, test_df_2 = np.split(merged_df.sample(frac=1, random_state=42),
                                     [int(.8*len(merged_df)), int(.9*len(merged_df))])

EPOCHS = 10
num_classes = 2
test_model = BertClassifier()
LR = 2e-5

cls_embeddings_with_text = train(test_model, train_df, val_df, LR, EPOCHS)
cls_embeddings = evaluate(test_model, test_df)

100%|██████████| 95/95 [00:12<00:00,  7.85it/s]


Epochs: 1 | Train Loss:  0.240 | Train Accuracy:  0.926 | Val Loss:  0.045 | Val Accuracy:  1.000


100%|██████████| 95/95 [00:12<00:00,  7.84it/s]


Epochs: 2 | Train Loss:  0.025 | Train Accuracy:  1.000 | Val Loss:  0.012 | Val Accuracy:  1.000


100%|██████████| 95/95 [00:12<00:00,  7.83it/s]


...

Epochs: 8 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.001 | Val Accuracy:  1.000


100%|██████████| 95/95 [00:12<00:00,  7.85it/s]


Epochs: 9 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.001 | Val Accuracy:  1.000


100%|██████████| 95/95 [00:12<00:00,  7.86it/s]


Epochs: 10 | Train Loss:  0.001 | Train Accuracy:  1.000 | Val Loss:  0.001 | Val Accuracy:  1.000
Test Accuracy:  0.944


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def top_k_indices(similarity_matrix, k):
    np.fill_diagonal(similarity_matrix, -np.inf)  # Exclude self-similarity
    flattened_indices = np.argsort(similarity_matrix.flatten())[::-1][:k]
    row_indices, col_indices = np.unravel_index(flattened_indices, similarity_matrix.shape)
    return row_indices, col_indices

def find_most_similar_tensors(tensors_list, k=10):
    """
    Find the k most similar tensors from a list of tensors using cosine similarity.

    Args:
    - tensors_list (list of torch.Tensor): A list containing tensors.
    - k (int): The number of most similar tensors to return.

    Returns:
    - most_similar_tensors (list of torch.Tensor): The k most similar tensors.
    """
    # Compute pairwise cosine similarities between tensors

    similarities = np.zeros((len(tensors_list), len(tensors_list)))
    for i, tensor1 in enumerate(tensors_list):
        for j, tensor2 in enumerate(tensors_list):
            cls_vector1_normalized = tensor1.cpu().detach().numpy() / np.linalg.norm(tensor1.cpu().detach().numpy())
            cls_vector2_normalized = tensor2.cpu().detach().numpy() / np.linalg.norm(tensor2.cpu().detach().numpy())
            similarity = cosine_similarity([cls_vector1_normalized], [cls_vector2_normalized])[0][0]
            similarities[i][j] = similarity
    row_indices, col_indices = top_k_indices(similarities, k)
    most_similar_tensors = []
    print("\nTop", k, "most similar tensors (excluding self-similarity):")
    for i in range(k):
      row_index = row_indices[i]
      col_index = col_indices[i]
      similarity_score = similarities[row_index, col_index]
      tensor = tensors_list[row_index]  # Get the tensor from the tensors_list
      print("Tensor Index:", row_index)
      print("Similarity Score:", similarity_score)
      #print("Tensor:", tensor)
      most_similar_tensors.append(tensor)
    return most_similar_tensors

# Example usage:
# Assuming tensors_list is a list containing 90 tensors
tensors_list_beginner =  [t[1] for t in cls_embeddings_with_text if t[2].item() == 0]
most_similar_tensors_beginner = find_most_similar_tensors(tensors_list_beginner, k=10) ## ici verifier pourquoi quand k = 2 je retourne 2 embeddings au lieu de 2 paires de embeddings - c'est possible qu'il y ai des regroupements dans les paires
# les paires les plus similaires ne sont pas representatives de donnees hors corpus
# novices et experts confondus -considerer la classe majoritaire- nombre expert / nombre total (0 etant le plus novice possible)- et proceder par majorite ou le ratio pour classifier- obtenir un embedding pour representer chaque classe, ce n'est possible avec les paires
#ou bien TODO : faire la moyenne de l'ensemble :
#mean_tensor_beginner = torch.mean(torch.stack(most_similar_tensors_beginner), dim=0)
#mean_tensor_expert = torch.mean(torch.stack(most_similar_tensors_expert), dim=0)
#integrer nouvelles donnnesm
#TODO : trouver un embedding de reference pour chaque classe


# Print the most similar tensors

tensors_list_expert =  [t[1] for t in cls_embeddings_with_text if t[2].item() == 1]
most_similar_tensors_expert = find_most_similar_tensors(tensors_list_expert, k=10)





Top 10 most similar tensors (excluding self-similarity):
Tensor Index: 249
Similarity Score: 0.9999986886978149
Tensor Index: 250
Similarity Score: 0.9999986886978149
Tensor Index: 458
Similarity Score: 0.9999899864196777
Tensor Index: 448
Similarity Score: 0.9999899864196777
Tensor Index: 403
Similarity Score: 0.9999896883964539
Tensor Index: 396
Similarity Score: 0.9999896883964539
Tensor Index: 438
Similarity Score: 0.9999752640724182
Tensor Index: 453
Similarity Score: 0.9999752640724182
Tensor Index: 344
Similarity Score: 0.9999671578407288
Tensor Index: 359
Similarity Score: 0.9999671578407288

Top 10 most similar tensors (excluding self-similarity):
Tensor Index: 273
Similarity Score: 0.9999998807907104
Tensor Index: 272
Similarity Score: 0.9999998807907104
Tensor Index: 390
Similarity Score: 0.9999986290931702
Tensor Index: 394
Similarity Score: 0.9999986290931702
Tensor Index: 363
Similarity Score: 0.9999985694885254
Tensor Index: 359
Similarity Score: 0.9999985694885254
Tens

In [ ]:
mean_tensor_beginner = torch.mean(torch.stack(most_similar_tensors_beginner), dim=0)
mean_tensor_expert = torch.mean(torch.stack(most_similar_tensors_expert), dim=0)

In [ ]:
tensors_list_beginner =  [t[1] for t in cls_embeddings_with_text if t[2].item() == 0]
tensors_list_expert =  [t[1] for t in cls_embeddings_with_text if t[2].item() == 1]
mean_tensor_beginner = torch.mean(torch.stack(tensors_list_beginner), dim=0)
mean_tensor_expert = torch.mean(torch.stack(tensors_list_expert), dim=0)

In [ ]:
mean_tensor_beginner.sum()

tensor(-14.5992, device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
mean_tensor_expert.sum()

tensor(-14.2722, device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
correct = 0
for t in cls_embeddings :
  #print(t[2].item())
  #print(t[1][0][:20])
  print("truth : ")

  if t[2].item() == 0 :
    truth = "beginner"
    print("beginner")
  else :
    truth = "expert"
    print("expert")

  cls_vector1_normalized = t[1][0].cpu().detach().numpy() / np.linalg.norm(t[1][0].cpu().detach().numpy())
  cls_vector2_normalized = mean_tensor_beginner.cpu().detach().numpy() / np.linalg.norm(mean_tensor_beginner.cpu().detach().numpy())

# Then, calculate cosine similarity
  similarity_beg = cosine_similarity([cls_vector1_normalized], [cls_vector2_normalized])[0][0]

  #print("Cosine Similarity beginner:", similarity)

# Assuming cls_vector1 and cls_vector2 are your CLS vectors
# First, normalize the vectors
  cls_vector1_normalized = t[1][0].cpu().detach().numpy() / np.linalg.norm(t[1][0].cpu().detach().numpy())
  cls_vector2_normalized = mean_tensor_expert.cpu().detach().numpy() / np.linalg.norm(mean_tensor_expert.cpu().detach().numpy())

# Then, calculate cosine similarity
  similarity_expert = cosine_similarity([cls_vector1_normalized], [cls_vector2_normalized])[0][0]

  #print("Cosine Similarity expert:", similarity)

  print("estimation : ")
  if similarity_beg > similarity_expert and truth == "beginner" :
    correct += 1
    print("beginner")
  elif similarity_beg < similarity_expert and truth == "expert" :
    correct += 1
    print("expert")

print("correct : ", correct/len(cls_embeddings),"\n")




truth : 
beginner
estimation : 
truth : 
beginner
estimation : 
truth : 
beginner
estimation : 
beginner
truth : 
expert
estimation : 
expert
truth : 
expert
estimation : 
expert
truth : 
expert
estimation : 
expert
truth : 
beginner
estimation : 
truth : 
beginner
estimation : 
beginner
truth : 
beginner
estimation : 
truth : 
expert
estimation : 
expert
truth : 
expert
estimation : 
expert
truth : 
expert
estimation : 
expert
truth : 
beginner
estimation : 
beginner
truth : 
beginner
estimation : 
beginner
truth : 
beginner
estimation : 
beginner
truth : 
expert
estimation : 
expert
truth : 
expert
estimation : 
expert
truth : 
expert
estimation : 
expert
correct :  0.7777777777777778 



TODO

In [ ]:
#TODO : assemble results in clear diagram, try with more or less then 10 closest vectors, try with other embeddings, read prof notes
# Get performance metrics and compare - have a baseline for comparaison
# ensure code is right
# ponderer avec le attention score mais utiliser un autre embedding tel que fast text?
-------
##summarize work, understand what i did
#try with the embeddings of tf the other methods (ensure the tokenization and reconstruction is right)
#try my final model
#understand the numbers - why are the sums all almost equal?

----
#EXTRACT ATTENTION SCORES

BROUILLON

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
import torch.optim as optim

class FullyConnectedNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FullyConnectedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x




In [ ]:
cls_embeddings_with_text[0][2]

tensor(0, device='cuda:0')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split([x[1].detach().cpu().numpy() for x in cls_embeddings_with_text], [x[2].detach().item() for x in cls_embeddings_with_text], test_size=0.2, random_state=42)

#X_train, y_train = [x[1].detach().item() for x in cls_embeddings_with_text], [x[2].detach().item() for x in cls_embeddings_with_text]
#X_val, y_val = [x[1].detach().item() for x in cls_embeddings_with_text], [x[2].detach().item() for x in cls_embeddings_with_text]
#= vzs_test


In [ ]:
# Convert NumPy arrays to PyTorch tensors

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)

# Define the FullyConnectedNN model
input_size = 768  # Size of the BERT [CLS] embeddings
hidden_size = 128  # Example hidden size
output_size = 2  # Example output size
model = FullyConnectedNN(input_size, hidden_size, output_size)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Validation

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_preds = torch.argmax(val_outputs, dim=1)
        val_accuracy = (val_preds == y_val_tensor).float().mean()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy.item():.4f}")


print("Training complete.")

Epoch 1/10, Loss: 0.6497, Val Loss: 0.2517, Val Accuracy: 1.0000
Epoch 2/10, Loss: 0.2580, Val Loss: 0.0979, Val Accuracy: 1.0000
Epoch 3/10, Loss: 0.1069, Val Loss: 0.0420, Val Accuracy: 1.0000
Epoch 4/10, Loss: 0.0517, Val Loss: 0.0199, Val Accuracy: 1.0000
Epoch 5/10, Loss: 0.0298, Val Loss: 0.0106, Val Accuracy: 1.0000
Epoch 6/10, Loss: 0.0202, Val Loss: 0.0062, Val Accuracy: 1.0000
Epoch 7/10, Loss: 0.0155, Val Loss: 0.0039, Val Accuracy: 1.0000
Epoch 8/10, Loss: 0.0130, Val Loss: 0.0027, Val Accuracy: 1.0000
Epoch 9/10, Loss: 0.0113, Val Loss: 0.0019, Val Accuracy: 1.0000
Epoch 10/10, Loss: 0.0103, Val Loss: 0.0015, Val Accuracy: 1.0000
Training complete.


<ipython-input-21-7eabae7df44b>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  X_train_tensor = torch.tensor(X_train, dtype=torch.float32)


In [ ]:
# Convert NumPy arrays to PyTorch tensors
def predict_final(vz) :
  X_test_tensor = torch.tensor(vz, dtype=torch.float32)


# Perform inference
  model.eval()
  with torch.no_grad():
      test_outputs = model(X_test_tensor)
      test_preds = torch.argmax(test_outputs)

# Convert predictions tensor to NumPy array
  test_preds_array = test_preds.numpy()

# Now, test_preds_array contains the predicted class labels for the test data
  print("Predictions:", test_preds_array)
  return test_preds_array

In [ ]:
count = 0
for t in cls_embeddings :
  print(t[2].item())
  predict_final(t[1].detach().cpu().numpy())
  if t[2].item() == t[1]:
    count +=1

  print("\n")
print(count)

0
Predictions: 1
tensor([[-4.6974e-01,  6.8680e-01, -1.7953e-01, -5.6552e-01,  5.9156e-01,
          1.9117e-01,  5.9148e-01,  4.6054e-01,  2.8598e-01, -3.1942e-01,
         -1.1019e+00,  1.1036e+00, -5.3062e-01,  1.1170e+00,  1.1479e+00,
          2.0014e-01, -7.7582e-02,  8.1851e-01, -7.6001e-01, -4.4952e-01,
         -1.1623e+00, -1.1406e-01,  1.5470e-01,  3.0982e-01,  1.6559e-01,
         -1.1347e-01, -8.5904e-01,  6.9815e-02, -7.8200e-01,  2.0219e-01,
          1.3745e-01, -5.3525e-01, -3.3343e-01, -1.2618e+00, -1.4071e-03,
         -1.1734e+00,  3.1290e-01,  2.0768e-01, -3.2258e-01, -1.4172e+00,
         -2.8726e-01,  8.4807e-01,  9.0239e-01, -1.6935e-01,  9.0878e-01,
          6.9108e-01, -1.3807e+00, -4.3345e-03, -2.0109e-01,  4.0040e-01,
         -1.1973e-01, -6.7484e-01,  4.4839e-01,  4.7229e-01, -3.0064e-01,
          7.8256e-01, -4.3261e-01, -1.1440e+00, -3.5507e-01,  4.9777e-01,
          8.6300e-01,  1.0937e+00,  3.9271e-01, -4.0738e-01,  5.2385e-01,
         -3.0339e-01,

RuntimeError: Boolean value of Tensor with more than one value is ambiguous